# Análisis comparativo y extendido del modelo YOLOv5 (2024)

## José Pablo Martínez A01275676
## Luis Arturo Rendón A01703572

In [ ]:
# Imports
import pandas as pd
import numpy as np
from io import StringIO
import os
from ace_tools import display_dataframe_to_user

In [ ]:
PRED_PATH = "analysis/RESULTADO_CSV.csv" 
GROUND_TRUGH   = "analysis/ground_truth.csv" 

In [ ]:
def parse_beds_str(beds_str: str) -> set[int]:
    """
    Convierte "0,1" -> {0,1}, "" -> set()
    """
    if pd.isna(beds_str) or str(beds_str).strip() == "":
        return set()
    return set(int(x) for x in str(beds_str).split(",") if x.strip().isdigit())

In [ ]:
df_pred = pd.read_csv(PRED_PATH)          # columnas: filename, n_cows, beds
df_gt   = pd.read_csv(GT_PATH)            # columnas: filename, n_cows, beds

# Renombramos para evitar colisiones
df_gt = df_gt.rename(columns={"n_cows": "true_n_cows", "beds": "true_beds"})

# Merge
df = pd.merge(df_gt, df_pred, on="filename", how="inner", validate="one_to_one")

In [ ]:
err_counts = []
correct_count_flags = []
correct_beds_flags = []

for idx, row in df.iterrows():
    # Conteo
    true_nc = int(row["true_n_cows"])
    pred_nc = int(row["n_cows"])
    err_counts.append(abs(pred_nc - true_nc))
    correct_count_flags.append(int(pred_nc == true_nc))

    # Camas
    true_beds = parse_beds_str(row["true_beds"])
    pred_beds = parse_beds_str(row["beds"])
    correct_beds_flags.append(int(pred_beds == true_beds))

# Añadimos columnas
df["err_count"] = err_counts
df["correct_count"] = correct_count_flags
df["correct_beds"] = correct_beds_flags

In [ ]:
N = len(df)
mae_count       = np.mean(err_counts)
accuracy_count  = np.mean(correct_count_flags) * 100
accuracy_beds   = np.mean(correct_beds_flags)  * 100

summary_dict = {
    "N imágenes"                  : N,
    "MAE conteo vacas"            : round(mae_count, 3),
    "Exactitud conteo (%)"        : round(accuracy_count, 2),
    "Exactitud camas (%)"         : round(accuracy_beds, 2),
}

In [ ]:
print("=== RESUMEN GLOBAL ===")
for k, v in summary_dict.items():
    print(f"{k:<25}: {v}")

# Mostrar detalle en tabla interactiva
display_dataframe_to_user("Comparación predicciones vs ground‑truth", df)